In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,  Seq2SeqTrainingArguments, Seq2SeqTrainer

model_id = "tdoehmen/t5-schemapile-fk"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
max_input_length = 1024

def generate_responses_as_json_t5(prompt):
    inputs = [prompt]
    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
    output = model.generate(**inputs, num_beams=8, do_sample=False, min_length=10, max_length=512)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    try:
        output_json = json.loads('{'+decoded_output+"}")
    except Exception as e:
        print('{'+decoded_output+"}")
        raise e
    return output_json

In [14]:
text = 'You are given the following SQL database tables: \nstaff(staff_id, staff_address_id, nickname, first_name, middle_name, last_name, date_of_birth, date_joined_staff, date_left_staff)\naddresses(address_id, line_1_number_building, city, zip_postcode, state_province_county, country)\nOutput a json string with the following schema {table, column, referencedTable, referencedColumn} that contains the foreign key relationship between the two tables.'
print(text)

You are given the following SQL database tables: 
staff(staff_id, staff_address_id, nickname, first_name, middle_name, last_name, date_of_birth, date_joined_staff, date_left_staff)
addresses(address_id, line_1_number_building, city, zip_postcode, state_province_county, country)
Output a json string with the following schema {table, column, referencedTable, referencedColumn} that contains the foreign key relationship between the two tables.


In [19]:
generate_responses_as_json_t5(text)

{'table': 'staff',
 'column': 'staff_address_id',
 'referencedTable': 'addresses',
 'referencedColumn': 'address_id'}